Example: Windowed Gaussian Process transition model with insect walk path data

Hyperparameter optimisation

- length scale and output variance of RBF kernel

In [314]:
# define constants

from sklearn.gaussian_process.kernels import RBF
import numpy as np
from datetime import datetime, timedelta

PRIOR_SCALE = 10
WINDOW_SIZE = 10
TIME_INTERVAL = timedelta(seconds=1)

KERNEL_LENGTH_SCALE = 10
KERNEL_OUT_VAR = 30

NOISE_COVAR = 2
PRIOR_COVAR = 1
num_steps = 100

np.random.seed(0)

In [315]:
# Define our 2D transition model
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
print(os.listdir(parent_dir))

from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               SlidingWindowGaussianProcess
from stonesoup.base import Property

# Sliding window GP with RBF kernel
class RBFKernelGP(SlidingWindowGaussianProcess):

    length_scale: float = Property()
    output_var: float = Property()

    def kernel(self, x, y, **kwargs):
        k = RBF(self.length_scale)
        return self.output_var * k(x,y)

transition_model = CombinedLinearGaussianTransitionModel([RBFKernelGP(window_size=WINDOW_SIZE, length_scale=KERNEL_LENGTH_SCALE, output_var=KERNEL_OUT_VAR),
                                                          RBFKernelGP(window_size=WINDOW_SIZE, length_scale=KERNEL_LENGTH_SCALE, output_var=KERNEL_OUT_VAR)])

['.circleci', '.codecov.yml', '.flake8', '.git', '.gitattributes', '.github', '.gitignore', '.git_archival.txt', '.pytest_cache', '.readthedocs.yml', '01_KalmanFilterTutorial (1).ipynb', '06_DataAssociation-MultiTargetTutorial.ipynb', 'binder', 'CITATION.cff', 'dataset_InsectFlightDynamics', 'docs', 'example_IWGP.py', 'example_WGP.py', 'InsectFlightDynamics_Sec1.ipynb', 'LICENSE', 'MANIFEST.in', 'notebooks', 'pyproject.toml', 'README.md', 'sec1.ipynb', 'sec1_cattle.ipynb', 'sec1_extra.ipynb', 'sec1_gp_generated.ipynb', 'sec1_gp_generated_single.ipynb', 'sec1_gp_prior.ipynb', 'sec1_hyp_optim.ipynb', 'sec2_igp_generated copy.ipynb', 'sec2_random.ipynb', 'sec3_dynamic_gp.ipynb', 'stonesoup', 'tractmethod', 'X and Y.', '__pycache__']


In [316]:
# Obtain ground truth coordinates from insect walk data

from get_coordinates import get_coordinates
import pandas as pd

# Load the CSV file into a DataFrame
file_path = 'Coordinate_second.csv'  # Replace with the actual file path
df = pd.read_csv(file_path)

# Extract all coordinates
type_name = 'Dmelanogaster' # 'Calocasiae' # 'Dmelanogaster'
movie_number = 1
fly_number = 5
x, y = get_coordinates(df, type_name, movie_number, fly_number)

In [317]:
# Convert raw coordiantes to ground truth path

from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.state import GaussianState, StateVector

start_time = datetime.now().replace(microsecond=0) 
curr_time = start_time + (WINDOW_SIZE-1) * TIME_INTERVAL  # first prediction at t = 5

truth = GroundTruthPath()

# Generate path
for k in range(num_steps+1):
    truth.append(GroundTruthState(np.concatenate((np.flip(x[k:WINDOW_SIZE+k]), np.flip(y[k:WINDOW_SIZE+k]))), timestamp=curr_time))
    curr_time += TIME_INTERVAL


In [318]:
# Simulate gaussian noise measurements
from stonesoup.types.detection import Detection
from stonesoup.models.measurement.linear import LinearGaussian

# Define measurement model
measurement_model = LinearGaussian(
    ndim_state=WINDOW_SIZE*2,
    mapping=(0, WINDOW_SIZE),
    noise_covar=np.array([[NOISE_COVAR, 0],
                          [0, NOISE_COVAR]])
    )

# Generate measurements
measurements = []
for state in truth:
    measurement = measurement_model.function(state, noise=True)
    measurements.append(Detection(measurement,
                                  timestamp=state.timestamp,
                                  measurement_model=measurement_model))

In [319]:
# Plot ground truth and simulated measurements

from stonesoup.plotter import AnimatedPlotterly

timesteps = [start_time + TIME_INTERVAL * i for i in range(num_steps + WINDOW_SIZE)]

plotter = AnimatedPlotterly(timesteps, tail_length=1)
plotter.plot_ground_truths(truth, [0, WINDOW_SIZE])
plotter.plot_measurements(measurements, [0, WINDOW_SIZE])
plotter.fig

In [320]:
# Initialise predictor and prior for kalman filtering

from stonesoup.predictor.kalman import KalmanPredictor

predictor = KalmanPredictor(transition_model)

prior_state_vector = np.zeros(WINDOW_SIZE*2)
prior_state_vector[:WINDOW_SIZE] += x[0]
prior_state_vector[WINDOW_SIZE:] += y[0]
prior = GaussianState(StateVector(prior_state_vector), np.identity(WINDOW_SIZE*2)*PRIOR_COVAR, timestamp=truth[0].timestamp)

In [321]:
from scipy.linalg import inv
from scipy.optimize import minimize
from stonesoup.types.prediction import GaussianStatePrediction
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.types.hypothesis import SingleHypothesis


measurement_model_gp_1d = LinearGaussian(
    ndim_state=WINDOW_SIZE,  # Assume 2D model
    mapping=([0]),  # Mapping measurement vector index to state index
    noise_covar=np.array([[NOISE_COVAR]])
    )

def optimise_length_scale(length_scale, output_var, prior: GaussianStatePrediction, pred_time, obs_matrix, obs_covar, y):
    # create model with length scale to be modified
    gp_model = RBFKernelGP(window_size=WINDOW_SIZE, length_scale=length_scale, output_var=output_var)
    predictor = KalmanPredictor(gp_model)

    # this needs to be a fcn of length_scale ie compute using RBF kernel
    def kalman_predictive_likelihood(length_scale, prior, pred_time, H, Cv, y):
        # print('Evaluating likelihood for length scale:', length_scale)

        # say prior at time t - 1
        # we pred for time t
        # we need measurements (y) from time = t-L+1 to t to cover entiree tstatevector at time t.

        log_likelihood = 0

        # modify length scale of model
        predictor.transition_model.length_scale = length_scale
        # print(length_scale)

        for i in range(1):  # do this for one timestep for now
            # y = prev_measurements[i]  # most recent last
            
            # prediction step
            # disabled predict_lru_cache for this. how to fix?
            # pass in length_scale through kwargs/normal arg instead of a model property? - can't have different length scales for x and y coors
            # use lru clear cache thing?

            state_pred = predictor.predict(prior, pred_time=pred_time, timestamp=prior.timestamp+timedelta(seconds=1))  # need to attach timestamp to prior and pass in as argument to calc time_interval
            m_pred = state_pred.state_vector
            P_pred = state_pred.covar
 
            S = H @ P_pred @ H.T + Cv  # Innovation covariance
            sign, logdet = np.linalg.slogdet(S)  # prevent numericla instabiltiy when inverting then taking log
            log_likelihood += -0.5 * (logdet +
                                      (y - H @ m_pred).T @ inv(S) @ (y - H @ m_pred))
            # print(log_likelihood)
            
        return -log_likelihood

    res = minimize(kalman_predictive_likelihood, x0=length_scale, 
                   args=(prior, pred_time, obs_matrix, obs_covar, y),
                   bounds=[(1,100)],  method="Nelder-Mead")

    return res.x[0]

def optimise_output_var(length_scale, output_var, prior: GaussianStatePrediction, pred_time, obs_matrix, obs_covar, y):
    # create model with length scale to be modified
    gp_model = RBFKernelGP(window_size=WINDOW_SIZE, length_scale=length_scale, output_var=output_var)
    predictor = KalmanPredictor(gp_model)

    def kalman_predictive_likelihood(output_var, prior, pred_time, H, Cv, y):
        log_likelihood = 0

        # modify length scale of model
        predictor.transition_model.output_var = output_var

        state_pred = predictor.predict(prior, pred_time=pred_time, timestamp=prior.timestamp+timedelta(seconds=1))  # need to attach timestamp to prior and pass in as argument to calc time_interval
        m_pred = state_pred.state_vector
        P_pred = state_pred.covar

        S = H @ P_pred @ H.T + Cv  # Innovation covariance
        sign, logdet = np.linalg.slogdet(S)  # prevent numericla instabiltiy when inverting then taking log
        log_likelihood += -0.5 * (logdet +
                                    (y - H @ m_pred).T @ inv(S) @ (y - H @ m_pred))
        # print(log_likelihood)
            
        return -log_likelihood

    res = minimize(kalman_predictive_likelihood, x0=output_var, 
                   args=(prior, pred_time, obs_matrix, obs_covar, y),
                   bounds=[(1,100)],  method="Nelder-Mead")

    return res.x[0]

In [322]:
# Kalman filtering: Prediction and update steps

from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track
from stonesoup.updater.kalman import KalmanUpdater

updater = KalmanUpdater(measurement_model)
track = Track()
track.append(prior)
L = WINDOW_SIZE
H = measurement_model_gp_1d.matrix()
Cv = measurement_model_gp_1d.covar()
length_scale_x, length_scale_y = KERNEL_LENGTH_SCALE, KERNEL_LENGTH_SCALE
output_var_x, output_var_y = KERNEL_OUT_VAR, KERNEL_OUT_VAR

for i in range(1, len(measurements)):
    measurement = measurements[i]
    prediction = predictor.predict(prior, timestamp=measurement.timestamp, pred_time=(measurement.timestamp-start_time))
    hypothesis = SingleHypothesis(prediction, measurement)
    post = updater.update(hypothesis)
    track.append(post)

    # update length scales
    current_x = measurement.state_vector[0]
    current_y = measurement.state_vector[1]
    pred_time = measurement.timestamp - start_time
    prediction_x = GaussianStatePrediction(prior.state_vector[:L], prior.covar[:L,:L], timestamp=measurement.timestamp)
    prediction_y = GaussianStatePrediction(prior.state_vector[L:], prior.covar[L:,L:], timestamp=measurement.timestamp)
    length_scale_x_new = optimise_length_scale(length_scale_x, output_var_x, prediction_x, pred_time, H, Cv, current_x)
    length_scale_y_new = optimise_length_scale(length_scale_y, output_var_y, prediction_y, pred_time, H, Cv, current_y)
    length_scale_x = 0.5*length_scale_x + 0.5*length_scale_x_new
    length_scale_y = 0.5*length_scale_y + 0.5*length_scale_y_new

    print('Updated length scales (x and y): ', length_scale_x, length_scale_y)

    # update output vars
    output_var_x_new = optimise_output_var(length_scale_x, output_var_x, prediction_x, pred_time, H, Cv, current_x)
    output_var_y_new = optimise_output_var(length_scale_y, output_var_y, prediction_y, pred_time, H, Cv, current_y)
    output_var_x = 0.5 * output_var_x + 0.5 * output_var_x_new
    output_var_y = 0.5 * output_var_y + 0.5 * output_var_y_new

    # Update the predictor's kernel parameters
    predictor.transition_model.model_list[0].length_scale = length_scale_x
    predictor.transition_model.model_list[0].output_variance = output_var_x
    predictor.transition_model.model_list[1].length_scale = length_scale_y
    predictor.transition_model.model_list[1].output_variance = output_var_y
    print('  Output variances (x, y): ', output_var_x, output_var_y)


    prior = track[-1]

# zero division error if prior.timestamp == measurement.timestamp

Updated length scales (x and y):  9.6044921875 9.603256225585938
  Output variances (x, y):  29.34375 30.75
Updated length scales (x and y):  11.46905568242073 9.415809854678809
  Output variances (x, y):  29.274706935711766 32.671875
Updated length scales (x and y):  10.341750551135922 11.572067091907504
  Output variances (x, y):  30.371793551332473 32.2634761730209
Updated length scales (x and y):  8.400841812751649 12.313402506343973
  Output variances (x, y):  30.846351377309965 31.809771039337793
Updated length scales (x and y):  7.431545464922927 16.93092844622298
  Output variances (x, y):  32.412673522595185 31.412151934962594
Updated length scales (x and y):  4.551186577511203 15.369281774596256
  Output variances (x, y):  32.21009431307897 32.179817073802866
Updated length scales (x and y):  3.148246647718703 14.162554670658746
  Output variances (x, y):  32.31075085780734 37.00011783213145
Updated length scales (x and y):  2.616634149209268 17.520672377708845
  Output varia

In [323]:
# Using Kalman Smoother is equivalent to using last elem in sliding window
# Kalman update step smoothes previous states

In [324]:
plotter.plot_tracks(track, [0, WINDOW_SIZE], track_label="Sliding window GP", uncertainty=False)
plotter.fig